In [1]:
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
import numpy as np
import pandas as pd
import torchvision.models as models 
from torchvision import transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
import matplotlib.pyplot as plt
import latent_plot as lp
from torch.autograd import Variable
from torch.optim import lr_scheduler
import seaborn as sns
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import copy
#from model import VAE

import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
from data_loader import *
#from model import convVAE
from sklearn.manifold import TSNE

In [2]:
#only on validation set
csv_path = '/home/vip/sayan-mandal/datasets/obj_criteria/20200619-objective.csv'
img_path = '/home/vip/sayan-mandal/datasets/obj_criteria/good_reduced/'

ResizeParam = 128
FD = FullDataLoader(csv_path, img_path,transform=transforms.Compose([transforms.Resize(ResizeParam), transforms.RandomHorizontalFlip(p=0.5) ,transforms.ToTensor()]))
FD2 = FullDataLoader(csv_path, img_path,transform=transforms.Compose([transforms.Resize(ResizeParam), transforms.ToTensor()]))

#random sampler
batch_size = 64
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(FD)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
val_indices =  indices[:split]

# Creating PT data samplers and loaders:
valid_sampler = SubsetRandomSampler(val_indices)

#train_loader = torch.utils.data.DataLoader(FD, batch_size=batch_size, sampler=train_sampler)
data_loader = torch.utils.data.DataLoader(FD2, batch_size=batch_size, sampler=valid_sampler)

In [3]:


class convVAE(nn.Module):
    def __init__(self, in_shape, n_latent):
        super(convVAE, self).__init__()
        self.in_shape = in_shape
        self.lin_shape = 16
        self.n_latent = n_latent
        c,h,w = in_shape
        self.z_dim = h//2**2 # receptive field downsampled 3 times
        self.encoder = nn.Sequential(
            nn.Conv2d(c, 16, kernel_size=3, stride=1, padding=1),  # 16, 128, 128
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),  # 32, 64, 64
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # 64, 64, 64
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.Conv2d(64, self.lin_shape, kernel_size=3, stride=2, padding=1),  # 16, 32, 32
            nn.BatchNorm2d(self.lin_shape),
            nn.LeakyReLU(),
            nn.Flatten()
        )
        self.fc1 = nn.Sequential(
            nn.Linear(self.lin_shape * (self.z_dim)**2, 2048*2),
            nn.BatchNorm1d(2048*2),
            nn.LeakyReLU(),
            nn.Linear( 2048*2,n_latent),
            nn.BatchNorm1d(n_latent),
            nn.LeakyReLU(),
        )
        self.fc21 = nn.Sequential(
            nn.Linear(n_latent, n_latent),
        )
        self.fc22 = nn.Sequential(
            nn.Linear(n_latent, n_latent),
        )

        self.fc3 = nn.Sequential(
            nn.Linear(n_latent, n_latent*2),
            nn.BatchNorm1d(n_latent*2),
            nn.LeakyReLU(),
            nn.Linear(n_latent*2, self.lin_shape * (self.z_dim)**2),
            nn.BatchNorm1d(self.lin_shape * (self.z_dim)**2),
            nn.LeakyReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(self.lin_shape, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def sample_z(self, mean, logvar):
        stddev = logvar.mul(0.5).cuda()
        noise = Variable(torch.randn(*mean.size())).cuda()
        return (noise * stddev) + mean

    def bottleneck(self, h):
        h = self.fc1(h)
        mean, logvar = self.fc21(h), self.fc22(h)
        z = self.sample_z(mean, logvar)
        return z, mean, logvar

    def encode(self, x):
        x = self.encoder(x)
        z, mean, logvar = self.bottleneck(x)
        return z, mean, logvar

    def decode(self, z):
        out = self.fc3(z)
        out = out.view(out.shape[0], self.lin_shape, self.z_dim,self.z_dim)
        out = self.decoder(out)
        return out

    def forward(self, x):
        z,mean, logvar = self.encode(x)
        out = self.decode(z)
        return out, z,mean, logvar

In [4]:
#vae model structure
nl = 2048
model = convVAE((3,128,128),nl).cuda()
model.load_state_dict(torch.load('/home/vip/sayan-mandal/vae-convmodels/dfcmmdvae_inp128_nl'+str(nl)+'_a1b1_1.torch'))
print(model)

convVAE(
  (encoder): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
    (9): Conv2d(64, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01)
    (12): Flatten(start_dim=1, end_dim=-1)
  )
  (fc1): Sequential(
    (0): Linear(in_features=16384, out_features=4096, bias=True)
    (1): BatchNo

In [5]:
for i, (data,l1,l2,l3,l4,l5,l6,l7,l8,l9,l10) in enumerate(data_loader):
    y, _,_ =  model.encode(Variable(data).cuda())
    y = y.cpu().detach().numpy()
    if i == 0:
        yd = y
        lbl1,lbl2,lbl3,lbl4,lbl5,lbl6,lbl7,lbl8,lbl9,lbl10 = l1.numpy(),l2.numpy(),l3.numpy(),l4.numpy(),l5.numpy(),l6.numpy(),l7.numpy(),l8.numpy(),l9.numpy(),l10.numpy()
    else:
        yd = np.append(yd,y,axis=0)
        lbl1 = np.append(lbl1,l1.numpy(), axis=0)
        lbl2= np.append(lbl2,l2.numpy(), axis=0)
        lbl3= np.append(lbl3,l3.numpy(), axis=0)
        lbl4= np.append(lbl4,l4.numpy(), axis=0)
        lbl5= np.append(lbl5,l5.numpy(), axis=0)
        lbl6= np.append(lbl6,l6.numpy(), axis=0)
        lbl7= np.append(lbl7,l7.numpy(), axis=0)
        lbl8= np.append(lbl8,l8.numpy(), axis=0)
        lbl9= np.append(lbl9,l9.numpy(), axis=0)
        lbl10 = np.append(lbl10,l10.numpy(), axis=0)

In [6]:
yd.shape

(1725, 2048)

In [7]:
lbl1 = lbl1.reshape(-1,1)
lbl2 = lbl2.reshape(-1,1)
lbl3 = lbl3.reshape(-1,1)
lbl4 = lbl4.reshape(-1,1)
lbl5 = lbl5.reshape(-1,1)
lbl6 = lbl6.reshape(-1,1)
lbl7 = lbl7.reshape(-1,1)
lbl8 = lbl8.reshape(-1,1)
lbl9 = lbl9.reshape(-1,1)
lbl10 = lbl10.reshape(-1,1)

In [8]:
cols = ['class','mdprob','psdprob','md_inf','md_sup','rnfl_g','rnfl_ti','rnfl_ni','rnfl_ts','rnfl_ns']
for i in range(2048):
    cols += ['ls_'+str(i)]

In [9]:
to_df = np.concatenate((lbl1,lbl2,lbl3,lbl4,lbl5,lbl6,lbl7,lbl8,lbl9,lbl10,yd),axis=1)

In [10]:
df = pd.DataFrame(data=to_df, columns=cols)

In [11]:
df

,class,mdprob,psdprob,md_inf,md_sup,rnfl_g,rnfl_ti,rnfl_ni,rnfl_ts,rnfl_ns,...,ls_2038,ls_2039,ls_2040,ls_2041,ls_2042,ls_2043,ls_2044,ls_2045,ls_2046,ls_2047
0,1.0,0.0,2.0,0.0,0.0,2.0,2.0,0.0,2.0,1.0,...,0.018631,-0.007593,-0.024147,0.034430,0.003646,-0.018710,-0.049644,0.004734,-0.016967,-0.035527
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.024525,-0.046875,-0.050352,0.003578,-0.048082,0.049293,-0.013819,0.046886,0.003239,0.017171
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.044626,0.037597,-0.027641,0.000538,-0.005712,0.079605,-0.062455,-0.032286,0.055590,0.017471
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.030876,-0.001552,0.023457,-0.064065,-0.040672,0.055315,0.042166,-0.000111,-0.023842,0.041734
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.014986,-0.030513,-0.049505,0.005362,0.001994,-0.003847,-0.019154,0.030369,0.007398,-0.031057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000302,0.038264,-0.021580,0.025392,0.110911,0.067325,0.029738,-0.097960,0.000255,-0.009319
1721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.017061,-0.070867,-0.015031,0.033257,0.016174,0.017757,-0.011400,0.004925,0.017435,0.037697
1722,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.004930,0.140802,0.097778,-0.033529,-0.003383,-0.025709,0.032572,-0.061165,0.030953,0.030013
1723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.008664,0.085284,0.020767,0.017655,-0.121980,0.042989,-0.018107,-0.005454,0.027331,0.099162


In [12]:
df.to_csv('latentdat.csv',index=False)